In [28]:
import matplotlib.pyplot as plt
import pandas as pd
import psycopg2

from dateutil import parser
from functools import reduce
from tensorflow.keras.layers import LSTM,Dense, Dropout, SpatialDropout1D
from tensorflow.keras.layers import Embedding
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

In [29]:
conn = psycopg2.connect(database='hackathon2020_dev', user="postgres", password="Allo123!", host="127.0.0.1", port="5432")

tweet_df = pd.read_sql_query('SELECT * FROM public.trump_tweets',con=conn,index_col=['id'])
tweet_df.head(2)

,tweetId,link,content,publishedAt,retweets,favorites,mentions,hashtags,created_at,updated_at,sentiment,moronic_index
id,,,,,,,,,,,,
52952,342312442300354561,https://twitter.com/realDonaldTrump/status/342...,Giving away money and revolutionizing crowdfun...,2013-06-05 11:10:20,32,18,@fundanything,None,2021-12-20 15:34:56.987241,2021-12-20 15:34:56.987241,1,None
52953,342313853767532545,https://twitter.com/realDonaldTrump/status/342...,@ tfore4 Our clubhouse on the North Sea. http:...,2013-06-05 11:15:57,4,1,None,None,2021-12-20 15:34:56.987241,2021-12-20 15:34:56.987241,0,None


In [30]:
spx_df = pd.read_csv('../db/stock_markets/sp500.csv')

spx_df.head(2)


,Date,Close/Last,Volume,Open,High,Low
0,12/17/2021,4620.64,--,4652.50,4666.70,4600.22
1,12/16/2021,4668.67,--,4719.13,4731.99,4651.89


In [31]:
tweet_df['impact_level'] = tweet_df['retweets'] * tweet_df['favorites']

impact_level = tweet_df['impact_level'].array
sentiment = tweet_df['sentiment'].array
published_at = tweet_df['publishedAt'].array

for i in range(len(impact_level)):
    if sentiment[i] == 0: impact_level[i] *= -1
    
tweet_df['impact_level'] = impact_level
tweet_df.head(5)

,tweetId,link,content,publishedAt,retweets,favorites,mentions,hashtags,created_at,updated_at,sentiment,moronic_index,impact_level
id,,,,,,,,,,,,,
52952,342312442300354561,https://twitter.com/realDonaldTrump/status/342...,Giving away money and revolutionizing crowdfun...,2013-06-05 11:10:20,32,18,@fundanything,None,2021-12-20 15:34:56.987241,2021-12-20 15:34:56.987241,1,None,576
52953,342313853767532545,https://twitter.com/realDonaldTrump/status/342...,@ tfore4 Our clubhouse on the North Sea. http:...,2013-06-05 11:15:57,4,1,None,None,2021-12-20 15:34:56.987241,2021-12-20 15:34:56.987241,0,None,-4
52954,342320095332294657,https://twitter.com/realDonaldTrump/status/342...,@ Judare So true!,2013-06-05 11:40:45,3,0,None,None,2021-12-20 15:34:56.987241,2021-12-20 15:34:56.987241,0,None,0
52955,342321095636049920,https://twitter.com/realDonaldTrump/status/342...,A-Rod has disgraced the blessed @ Yankees orga...,2013-06-05 11:44:43,665,386,@Yankees,None,2021-12-20 15:34:56.987241,2021-12-20 15:34:56.987241,1,None,256690
52956,342341507421990913,https://twitter.com/realDonaldTrump/status/342...,"Kern County, CA has secured $1.2B for windfarm...",2013-06-05 13:05:50,48,16,None,None,2021-12-20 15:34:56.987241,2021-12-20 15:34:56.987241,1,None,768


In [38]:
data_by_date = {}

for i in range(len(impact_level)):
    date = published_at[i].strftime('%Y-%m-%d')

    if date not in data_by_date.keys(): data_by_date[date] = {'count': 0, 'impact_levels': [], 'average_impact': 0.0}
    data_by_date[date]['count'] += 1
    data_by_date[date]['impact_levels'].append(impact_level[i])
    data_by_date[date]['average_impact'] = round(reduce(lambda a, b: a + b, data_by_date[date]['impact_levels']) / data_by_date[date]['count'])


In [41]:
update_sql = []

for key in data_by_date.keys():
    update_sql.append('insert into public.trump_tweets_daily_stats ("date", average_impact_level, tweet_count, created_at, updated_at) VALUES(\'{}\', {}, {}, now(), now());'.format(key, data_by_date[key]['average_impact'], data_by_date[key]['count']))

with open('../db/update_data/update_daily_stats.sql', 'w') as output:
    output.write('\n'.join(update_sql))
    
print(update_sql[0])

insert into public.trump_tweets_daily_stats ("date", average_impact_level, tweet_count, created_at, updated_at) VALUES('2013-06-05', 1668927, 44, now(), now());
